In [ ]:
import os
import ntpath
import cv2
import numpy as np
from PIL import Image, ImageChops, ImageDraw, ImageOps, ImageFilter, ImageStat, ImageEnhance 
from skimage import io, color, exposure, transform
import imutils
import argparse
import sys
import matplotlib
import matplotlib.pyplot as plt
import glob
import math

import shutil
from datetime import datetime
import random
import blend_modes

original = True # Whether we are using the original exposure_manipulation code or not

In [ ]:
# Return a list with paths of all the files in the directory
def load_paths(directory):
    paths = []
    for filename in os.listdir(directory): # Retrieve names of files in directory
        # Concatenate filename with directory path
        if not filename.startswith('.'):  # Ignore hidden files
            paths.append(directory + "/" + filename)
    return paths

In [ ]:
# Rescale and pad the source image with whitespace to be a perfect square of fixed width
def scale_image(image_path):
    width = 450

    # https://jdhao.github.io/2017/11/06/resize-image-to-square-with-padding/
    # Resizing the image
    img = Image.open(image_path)
    old_size = img.size # old_size is in (width, height) format
    ratio = float(width) / max(old_size)
    new_size = tuple([int(x * ratio) for x in old_size])
    img = img.resize(new_size, Image.ANTIALIAS)

    # Padding the image with whitespace
    delta_w = width - new_size[0]
    delta_h = width - new_size[1]
    padding = (delta_w // 2, delta_h // 2, delta_w - (delta_w // 2), delta_h - (delta_h // 2))
    new_img = ImageOps.expand(img, padding, (255,255,255))
    return new_img

In [ ]:
# Based on Alexandros Stergiou's "find_borders()"
# Returns an alpha channel that matches the white background
def create_alpha(img, alpha_channel):
    # Read and decode the image contents for pixel access
    pix = img.load()

    # Note PIL indexes [x,y] while OpenCV indexes [y,x]
    # alpha_channel must be indexed [y,x] to merge with OpenCV channels later

    min = 200
    width, height = img.size
    # Loop through each row of the image
    for y in range(0, height):
        # First loop left to right
        for x in range(0, width, 1):
            # Retrieve a tuple with RGB values for this pixel
            rgb = pix[x,y]
            # Make transparent if the pixel is white (or light enough)
            if rgb[0] >= min and rgb[1] >= min and rgb[2] >= min:
                alpha_channel[y,x] = 0
            # If pixel is not white then we've hit the sign so break out of loop
            else:
                break

        # Then loop backwards, right to left
        for x in range(width-1, -1, -1):
            rgb = pix[x,y]
            if rgb[0] >= min and rgb[1] >= min and rgb[2] >= min:
                alpha_channel[y,x] = 0
            else:
                break

    return alpha_channel

In [ ]:
# Based on Alexandros Stergiou's "manipulate_images()"
# Delete the white background from the original sign
def delete_background(image_path, save_path):
    # Open the image using PIL (don't read contents yet)
    img = Image.open(image_path)
    img = img.convert('RGB')  # Does this have any effect??

    # Open the image again using OpenCV and split into its channels
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    channels = cv2.split(image)

    # Create a fully opaque alpha channel, same dimentions and dtype as the image
    # Create_alpha() modifies it to make the white background transparent
    alpha_channel = np.ones(channels[0].shape, dtype=channels[0].dtype) * 255
    alpha_channel = create_alpha(img, alpha_channel)

    # Merge alpha channel into original image
    image_RGBA = cv2.merge((channels[0], channels[1], channels[2], alpha_channel))

    
    cv2.imwrite(save_path, image_RGBA)

    img.close()

In [ ]:
# Make white backgrounds transparent
input_dir = "Traffic_Signs_Templates/1_Images"
output_dir = "Traffic_Signs_Templates/2_Processed_Images"
# Create the output directory if it doesn't exist already
if (not os.path.exists(output_dir)):
    os.mkdir(output_dir)

# Get start time to measure the number of clock-cycles it takes
start = cv2.getTickCount()

paths = load_paths(input_dir)
for path in paths:
    img = scale_image(path) # Rescale the image

    # Remove the extension and save as a png
    _, filename = ntpath.split(path)
    name, _ = filename.rsplit('.', 1)
    save_path = output_dir + "/" + name + ".png"
    img.save(save_path)

    delete_background(save_path, save_path) # Overwrite the newly rescaled image

# Get end time
end = cv2.getTickCount()
cycles = (end - start) / cv2.getTickFrequency()
print(">>> This took:", cycles, "clock-cycles.")

In [ ]:
# Overlays foreground image on background image, keeping transparency
# x1, y1: top-left coordinate to place foreground
# Image overlay code credit to fireant:
# https://stackoverflow.com/questions/14063070/overlay-a-smaller-image-on-a-larger-image-python-opencv
# RGB to RGBA conversion credit to kaanoner:
# https://stackoverflow.com/questions/32290096/python-opencv-add-alpha-channel-to-rgb-image
# https://docs.opencv.org/3.4.2/d3/df2/tutorial_py_basic_ops.html used as reference
def overlay(fg, bg, x1 = -1, y1 = -1):
    # If the background doesn't have an alpha channel, add one, but keep the entire image opaque
    # Code from kaanoner
    if len(cv2.split(bg)) == 3:
        bg = cv2.cvtColor(bg, cv.COLOR_RGB2RGBA)
        bg[:, :, 3] = 255
    # Make a copy of the background for making changes
    newImage = bg.copy()

    # Retrieve image dimentions
    heightFG, widthFG, _ = fg.shape  # Discard channel
    heightBG, widthBG, _ = bg.shape

    # If either of the coordinates were omitted, calculate start/end positions
    # using the difference in image size, centring foreground on background
    if x1 == -1 or y1 == -1:
        # Calculate start coordinates 
        x1 = (widthBG - widthFG) // 2    # floor division to truncate as
        y1 = (heightBG - heightFG) // 2  # coordinates don't need to be exact
    # Calculate end coordinates
    x2 = x1 + widthFG
    y2 = y1 + heightFG

    ### Start of code from fireant ###
    # Retrieve an array of alpha values from the foreground image
    # Divide by 255 to get values between 0.0 and 1.0
    alpha = fg[:, :, 3] / 255.0
    beta = 1.0 - alpha

    # Loop over BGR channels (but not alpha)
    for ch in range(0, 3):
        newImage[y1:y2, x1:x2, ch] = (alpha * fg[:, :, ch] +
                                      beta * newImage[y1:y2, x1:x2, ch])
    ### End of code from fireant ###

    return newImage

In [ ]:
# Resize the first image if it is larger than the second image
def resize(img1, img2):
    # retrieve dimentions of both images
    height1, width1, _ = img1.shape  # discard channel
    height2, width2, _ = img2.shape

    # resize if foreground is taller than background
    if height1 > height2:
        img1 = resizeOnHeight(img1, height2)
        height1, width1, _ = img1.shape  #re-retrieve dimentions
    # or wider
    if width1 > width2:
        img1 = resizeOnWidth(img1, width2)

    return img1, img2

def resizeOnHeight(img, newHeight):
    oldHeight, oldWidth, _ = img.shape  # discard channel
    newWidth = int( round( newHeight / oldHeight * oldWidth ) )
    img = cv2.resize(img, (newHeight, newWidth))
    return img

def resizeOnWidth(img, newWidth):
    oldWidth, oldHeight, _ = img.shape  # discard channel
    newHeight = int( round( newWidth / oldWidth * oldHeight ) )
    img = cv2.resize(img, (newWidth, newHeight))
    return img

In [ ]:
# calculates the ratio of opacity in first image compared to second image
def calcRatio(fg, bg):
    fgPixels = countPixels(fg)  # number of non-transparent pixels in graffiti
    bgPixels = countPixels(bg)  # total number of pixels, dependent on sign shape
    ratio = fgPixels / bgPixels

    return ratio

# return the number of non-transparent pixels in the imported image
def countPixels(img):
    count = 0
    split = cv2.split(img)
    if len(split) is 4:  # only proceed if the image has an alpha channel
        alpha = split[3]
        # loop through alpha channel, incrementing count if not opaque
        for ii in range(0, len(alpha)):
            for jj in range(0, len(alpha[0])):
                if alpha[ii][jj] > 0:
                    count += 1
    return count

In [ ]:
# Calculates the ratio of changed opaque pixels between two versions of the same image for each quadrant
def calcQuadrantDiff(new, original):
    height, width, ch = original.shape
    centre_x = int(width / 2)
    centre_y = int(height / 2)

    # Dividing the two images into quadrants
    new_I   = new[0:centre_y, centre_x:width]
    new_II  = new[0:centre_y, 0:centre_x]
    new_III = new[centre_y:height, 0:centre_x]
    new_IV  = new[centre_y:height, centre_x:width]
    original_I   = original[0:centre_y, centre_x:width]
    original_II  = original[0:centre_y, 0:centre_x]
    original_III = original[centre_y:height, 0:centre_x]
    original_IV  = original[centre_y:height, centre_x:width]

    ratio_I   = countDiffPixels(new_I, original_I) / countPixels(original_I)
    ratio_II  = countDiffPixels(new_II, original_II) / countPixels(original_II)
    ratio_III = countDiffPixels(new_III, original_III) / countPixels(original_III)
    ratio_IV  = countDiffPixels(new_IV, original_IV) / countPixels(original_IV)
    return [ratio_I, ratio_II, ratio_III, ratio_IV]

# Modified version of countPixels(img) which counts how many opaque pixels are different between two images
def countDiffPixels(new, original):
    count = 0
    split = cv2.split(original)
    if len(split) is 4:
        alpha = split[3]
        for ii in range(0, len(new)):
            for jj in range(0, len(new[0])):
                if alpha[ii][jj] > 0: # The pixel in the original image must have been opaque
                    if np.any(new[ii][jj] != original[ii][jj]): # Check for any changes in the pixel
                        count += 1
    return count

In [ ]:
damage_types = ["_ORIGINAL", "_QUADRANT", "_BIG_HOLE", "_HOLES",# "_YELLOW",
                "_GRAFFITI"] #Used for SGTSD

#Remove trailing zeroes and prevent automatic scientific exponent notation https://stackoverflow.com/a/37736333/12350950
def floatToString(num):
    return ("%.6f" % num).rstrip("0").rstrip(".")

def damage_images(paths):
    for image_path in paths:
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        img = img.astype('uint8')
        height, width, ch = img.shape
        centre_x = int(width / 2)
        centre_y = int(height / 2)
    
        #Creating Mask
        sign = img.copy() 
        alpha = sign[:, :, 3] #Extract the alpha channel from sign
        _, alpha = cv2.threshold(alpha, 5, 255, cv2.THRESH_BINARY) #Remove gradual transparency
        mask = alpha
        
        #Info for File Writing
        ii = 0 #Used for label filenames
        path = "Traffic_Signs_Templates/3_Damaged_Images/"
        head, tail = ntpath.split(image_path)
        title, extension = tail.split('.')

        direct = "SGTSD/Labels/"
        head, tail = image_path.split('.')
        name = head.split('/')
        c_num = name[-1] #name[-1] is the sign's name, which should be a class number
        if (not os.path.exists(direct + c_num)):
            os.makedirs(direct + c_num)
        
        
        #ORIGINAL
        if "_ORIGINAL" in damage_types:
            dmg = img
            
            dmg_values = calcQuadrantDiff(dmg, img)
            values = [floatToString(round(num, 6)) for num in dmg_values] #Convert the values into decimal strings
            text_file = open(direct + c_num + "/" + c_num + "_" + str(ii) + ".txt", "w")
            text_file.write(f"{values[0]} {values[1]} {values[2]} {values[3]}")
            ii += 1
            cv2.imwrite(path + title + "_ORIGINAL.png", dmg)
        
        
        #QUADRANT
        if "_QUADRANT" in damage_types:
            quadrant = mask.copy()
            quad_num = random.randint(1, 4) #Selecting which quadrant it will be
            #Removing the quadrant: -1 offset is necessary to avoid damaging part of a wrong quadrant
            if quad_num is 1:
                cv2.rectangle(quadrant, (width, 0), (centre_x, centre_y - 1), (0,0,0), -1)
            if quad_num is 2:
                cv2.rectangle(quadrant, (0, 0), (centre_x - 1, centre_y - 1), (0,0,0), -1)
            if quad_num is 3:
                cv2.rectangle(quadrant, (0, height), (centre_x - 1, centre_y), (0,0,0), -1)
            if quad_num is 4:
                cv2.rectangle(quadrant, (width, height), (centre_x, centre_y), (0,0,0), -1)
            dmg1 = cv2.bitwise_and(img, img, mask=quadrant)
            
            dmg_values = calcQuadrantDiff(dmg1, img)
            values = [floatToString(round(num, 6)) for num in dmg_values] #Convert the values into decimal strings
            text_file = open(direct + c_num + "/" + c_num + "_" + str(ii) + ".txt", "w")
            text_file.write(f"{values[0]} {values[1]} {values[2]} {values[3]}")
            ii += 1
            cv2.imwrite(path + title + "_QUADRANT.png", dmg1)
        
        
        #BIG HOLE
        if "_BIG_HOLE" in damage_types:
            big_hole = mask.copy()
            angle = random.randint(0, 359) #angle from the x axis counter-clockwise through quadrant I
            r = int(2 * height / 5) #radius
            rad = -(angle * math.pi / 180)
            x = centre_x + int(r * math.cos(rad)) #x coordinate of centre
            y = centre_y + int(r * math.sin(rad)) #y coordinate of centre
            cv2.circle(big_hole, (x,y), r, (0,0,0), -1)
            dmg2 = cv2.bitwise_and(img, img, mask=big_hole)

            dmg_values = calcQuadrantDiff(dmg2, img)
            values = [floatToString(round(num, 6)) for num in dmg_values] #Convert the values into decimal strings
            text_file = open(direct + c_num + "/" + c_num + "_" + str(ii) + ".txt", "w")
            text_file.write(f"{values[0]} {values[1]} {values[2]} {values[3]} angle={angle}")
            ii += 1
            cv2.imwrite(path + title + "_BIG_HOLE.png", dmg2)
        #FIXME: Images need to be square, with sign centred
        
        
        #RANDOMISED BULLET HOLES
        if "_HOLES" in damage_types:
            bullet_holes = mask.copy()
            painted = img.copy() #Another copy of original sign to draw 'flaking paint' from holes on
            numHoles = random.randint(7, 30)
            for x in range(numHoles):
                size = random.randint(2, 12) #Random hole size
                h_x = random.randint(0, height) #Random hole position
                h_y = random.randint(0, height)
                c = random.randint(0, 150) #How black/grey the 'hole' is if it didn't penetrate
                s = random.uniform(1.6, 2.2) #Random annulus size

                C = 200 #Colour of damaged 'paint' outer annulus
                cv2.circle(painted, (h_x, h_y), int(size * s), (C,C,C,255), -1) #Hole annulus to represent damaged 'paint'
                if (size < 6): #Did the bullet penetrate through the sign?
                    cv2.circle(painted, (h_x, h_y), size, (c,c,c,255), -1) #If not, grey out rather than make transparent
                else:
                    cv2.circle(bullet_holes, (h_x, h_y), size, (0,0,0), -1)
            dmg3 = cv2.bitwise_and(painted, painted, mask=bullet_holes)
            
            dmg_values = calcQuadrantDiff(dmg3, img)
            values = [floatToString(round(num, 6)) for num in dmg_values] #Convert the values into decimal strings
            text_file = open(direct + c_num + "/" + c_num + "_" + str(ii) + ".txt", "w")
            text_file.write(f"{values[0]} {values[1]} {values[2]} {values[3]} holes={numHoles}")
            ii += 1
            cv2.imwrite(path + title + "_HOLES.png", dmg3)
        
        
        #TINTED YELLOW
        if "_YELLOW" in damage_types:
            yellow = np.zeros((height,width,ch), np.uint8)
            yellow[:,:] = (0,210,210,255)
            dmg4 = cv2.bitwise_and(img, yellow)
            
            #TODO: Use quadrant pixel difference ratio or some other damage metric for labelling?
            cv2.imwrite(path + title + "_YELLOW.png", dmg4)
            
            
        #GRAFFITI
        if "_GRAFFITI" in damage_types:
            graffiti = "graffiti_black.png"
            fg = cv2.imread("Traffic_Signs_Templates/Graffiti/" + graffiti, cv2.IMREAD_UNCHANGED)
            bg = img.copy()
            fg, bg = resize(fg, bg) #Correct graffiti size to fit onto sign
            dmg5 = overlay(fg, bg)
            ratio = calcRatio(fg, bg) #Calculate the ratio of obscurity
            
            #Ratio is displayed in filename as a float without the first "0." part
            #TODO: check ratio == 1.0 edge case
            #cv2.imwrite(path + title + "_GRAFFITI-0," + str(int(round(ratio, 3) * 10**3)) + ".png", dmg5)
            dmg_values = calcQuadrantDiff(dmg5, img)
            values = [floatToString(round(num, 6)) for num in dmg_values] #Convert the values into decimal strings
            text_file = open(direct + c_num + "/" + c_num + "_" + str(ii) + ".txt", "w")
            text_file.write(f"{values[0]} {values[1]} {values[2]} {values[3]} graffiti='{graffiti}'")
            ii += 1
            cv2.imwrite(path + title + "_GRAFFITI.png", dmg5)
            
        
        #FADE (doesn't work as a damage type with Stergiou's exposure_manipulation function - also outdated)
#         if "_FADE" in damage_types:
#             #Retrieve alpha data from original image
#             splitImg = cv2.split(img)
#             if len(splitImg) is 4:
#                 alphaData = splitImg[3]
#                
#             for ii in range(5):
#                 dmg6 = img.copy()
#                 alpha = 1 - (ii * 0.19)
#                 beta = (ii + 1) * 40
#                 cv2.convertScaleAbs(img, dmg6, alpha, beta) #Scale the contrast and brightness
#                 dmg6[:, :, 3] = alphaData
#                
#                 cv2.imwrite(path + title + "_FADE-" + str(ii) + ".png", dmg6)
            
        
        #TODO: CRACKS (thin crack lines across the sign?)
        #TODO: MISSING SECTIONS (missing polygon sections on edges of sign?)

In [ ]:
#Creating damaged sign templates based on processed images of original templates
dmg_dir = "Traffic_Signs_Templates/3_Damaged_Images"
if os.path.exists(dmg_dir):
    shutil.rmtree(dmg_dir) #Clear out any old damage variations
os.mkdir(dmg_dir) #Recreate the directory for damaged images
paths = load_paths("Traffic_Signs_Templates/2_Processed_Images")
damage_images(paths)

In [ ]:
def img_transform(paths):
    for image_path in paths:
        img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
        rows,cols,ch = img.shape
        t = []
        for i in range(0,100):
            t.append(i)

        #FORWARD FACING
        dst = img
        
        #EAST FACING
        pts1 = np.float32([[cols/10,rows/10],[cols/2,rows/10],[cols/10,rows/2]])
        pts2 = np.float32([[cols/5,rows/5],[cols/2,rows/8],[cols/5,rows/1.8]])
        M = cv2.getAffineTransform(pts1,pts2)
        dst1 = cv2.warpAffine(img,M,(cols,rows))
        
        #NORTH-WEST FACING
        pts3 = np.float32([[cols*9/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts4 = np.float32([[cols*4.5/5,rows/5],[cols/2,rows/8],[cols*4.5/5,rows/1.8]])
        M = cv2.getAffineTransform(pts3,pts4)
        dst2 = cv2.warpAffine(img,M,(cols,rows))
        
        #LEFT TILTED FORWARD FACING
        pts5 = np.float32([[cols/10,rows/10],[cols/2,rows/10],[cols/10,rows/2]])
        pts6 = np.float32([[cols/12,rows/6],[cols/2.1,rows/8],[cols/10,rows/1.8]])
        M = cv2.getAffineTransform(pts5,pts6)
        dst3 = cv2.warpAffine(img,M,(cols,rows))
        
        #RIGHT TILTED FORWARD FACING
        pts7 = np.float32([[cols*9/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts8 = np.float32([[cols*10/12,rows/6],[cols/2.2,rows/8],[cols*8.4/10,rows/1.8]])
        M = cv2.getAffineTransform(pts7,pts8)
        dst4 = cv2.warpAffine(img,M,(cols,rows))
        
        #WEST FACING
        pts9 = np.float32([[cols/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts10 = np.float32([[cols/9.95,rows/10],[cols/2.05,rows/9.95],[cols*9/10,rows/2.05]])
        M = cv2.getAffineTransform(pts9,pts10)
        dst5 = cv2.warpAffine(img,M,(cols,rows))
        
        #RIGHT TILTED FORWARD FACING
        pts11 = np.float32([[cols*9/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts12 = np.float32([[cols*9/10,rows/10],[cols/2,rows/9],[cols*8.95/10,rows/2.05]])
        M = cv2.getAffineTransform(pts11,pts12)
        dst6 = cv2.warpAffine(img,M,(cols,rows))
        
        #FORWARD FACING W/ DISTORTION
        pts13 = np.float32([[cols/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts14 = np.float32([[cols/9.8,rows/9.8],[cols/2,rows/9.8],[cols*8.8/10,rows/2.05]])
        M = cv2.getAffineTransform(pts13,pts14)
        dst7 = cv2.warpAffine(img,M,(cols,rows))
        
        #FORWARD FACING W/ DISTORTION 2
        pts15 = np.float32([[cols/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts16 = np.float32([[cols/11,rows/10],[cols/2.1,rows/10],[cols*8.5/10,rows/1.95]])
        M = cv2.getAffineTransform(pts15,pts16)
        dst8 = cv2.warpAffine(img,M,(cols,rows))
        
        #FORWARD FACING W/ DISTORTION 3
        pts17 = np.float32([[cols/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts18 = np.float32([[cols/11,rows/11],[cols/2.1,rows/10],[cols*10/11,rows/1.95]])
        M = cv2.getAffineTransform(pts17,pts18)
        dst9 = cv2.warpAffine(img,M,(cols,rows))
        
        #FORWARD FACING W/ DISTORTION 4
        pts19 = np.float32([[cols*9.5/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts20 = np.float32([[cols*9.35/10,rows/9.99],[cols/2.05,rows/9.95],[cols*9.05/10,rows/2.03]])
        M = cv2.getAffineTransform(pts19,pts20)
        dst10 = cv2.warpAffine(img,M,(cols,rows))
         
        #FORWARD FACING W/ DISTORTION 5
        pts21 = np.float32([[cols*9.5/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts22 = np.float32([[cols*9.65/10,rows/9.95],[cols/1.95,rows/9.95],[cols*9.1/10,rows/2.02]])
        M = cv2.getAffineTransform(pts21,pts22)
        dst11 = cv2.warpAffine(img,M,(cols,rows))
        
        #FORWARD FACING W/ DISTORTION 6
        pts23 = np.float32([[cols*9.25/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts24 = np.float32([[cols*9.55/10,rows/9.85],[cols/1.9,rows/10],[cols*9.3/10,rows/2.04]])
        M = cv2.getAffineTransform(pts23,pts24)
        dst12 = cv2.warpAffine(img,M,(cols,rows))
        
        #SHRINK 1
        pts25 = np.float32([[cols*9/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts26 = np.float32([[cols*8/10,rows/10],[cols*1.34/3,rows/10.5],[cols*8.24/10,rows/2.5]])
        M = cv2.getAffineTransform(pts25,pts26)
        dst13 = cv2.warpAffine(img,M,(cols,rows))
        
        #SHRINK 2
        pts27 = np.float32([[cols*9/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts28 = np.float32([[cols*8.5/10,rows*3.1/10],[cols/2,rows*3/10],[cols*8.44/10,rows*1.55/2.5]])
        M = cv2.getAffineTransform(pts27,pts28)
        dst14 = cv2.warpAffine(img,M,(cols,rows))
        
        #FORWARD FACING W/ DISTORTION 7
        pts29 = np.float32([[cols*9/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts30 = np.float32([[cols*8.85/10,rows/9.3],[cols/1.9,rows/10.5],[cols*8.8/10,rows/2.11]])
        M = cv2.getAffineTransform(pts29,pts30)
        dst15 = cv2.warpAffine(img,M,(cols,rows))
        
        #FORWARD FACING W/ DISTORTION 8
        pts31 = np.float32([[cols*9/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts32 = np.float32([[cols*8.75/10,rows/9.1],[cols/1.95,rows/8],[cols*8.5/10,rows/2.05]])
        M = cv2.getAffineTransform(pts31,pts32)
        dst16 = cv2.warpAffine(img,M,(cols,rows))
        
        #FORWARD FACING W/ DISTORTION 9
        pts33 = np.float32([[cols*9/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts34 = np.float32([[cols*8.75/10,rows/9.1],[cols/1.95,rows/9],[cols*8.5/10,rows/2.2]])
        M = cv2.getAffineTransform(pts33,pts34)
        dst17 = cv2.warpAffine(img,M,(cols,rows))
        
        #FORWARD FACING W/ DISTORTION 10
        pts35 = np.float32([[cols*9/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts36 = np.float32([[cols*8.75/10,rows/8],[cols/1.95,rows/8],[cols*8.75/10,rows/2]])
        M = cv2.getAffineTransform(pts35,pts36)
        dst18 = cv2.warpAffine(img,M,(cols,rows))
        
        #FORWARD FACING W/ DISTORTION 11
        pts37 = np.float32([[cols*9/10,rows/10],[cols/2,rows/10],[cols*9/10,rows/2]])
        pts38 = np.float32([[cols*8.8/10,rows/7],[cols/1.95,rows/7],[cols*8.8/10,rows/2]])
        M = cv2.getAffineTransform(pts37,pts38)
        dst19 = cv2.warpAffine(img,M,(cols,rows))
        
        head, tail = ntpath.split(image_path)
        title, extension = tail.split('.')
        name = title.split('_') #name[0] gives us the sign number
        
        path = "Traffic_Signs_Templates/4_Transformed_Images/" + name[0] + "/" + title + "/"
        cv2.imwrite(path + str(1) + ".png", dst)
        cv2.imwrite(path + str(2) + ".png", dst1)
        cv2.imwrite(path + str(3) + ".png", dst2)
        cv2.imwrite(path + str(4) + ".png", dst3)
        cv2.imwrite(path + str(4) + ".png", dst4)
        cv2.imwrite(path + str(5) + ".png", dst5)
        cv2.imwrite(path + str(6) + ".png", dst6)
        cv2.imwrite(path + str(7) + ".png", dst7)
        cv2.imwrite(path + str(8) + ".png", dst8)
        cv2.imwrite(path + str(9) + ".png", dst9)
        cv2.imwrite(path + str(10) + ".png", dst10)
        cv2.imwrite(path + str(11) + ".png", dst11)
        cv2.imwrite(path + str(12) + ".png", dst12)
        cv2.imwrite(path + str(13) + ".png", dst13)
        cv2.imwrite(path + str(14) + ".png", dst14)
        cv2.imwrite(path + str(15) + ".png", dst15)
        cv2.imwrite(path + str(16) + ".png", dst16)
        cv2.imwrite(path + str(17) + ".png", dst17)
        cv2.imwrite(path + str(18) + ".png", dst18)
        cv2.imwrite(path + str(19) + ".png", dst19)
        plt.show()

In [ ]:
directory1 = 'Traffic_Signs_Templates/1_Images'
directory2 = 'Traffic_Signs_Templates/3_Damaged_Images' #Changed from /2_Processed_Images
if (not os.path.exists("Traffic_Signs_Templates/4_Transformed_Images")):
    for path1 in load_paths(directory1):
        head, tail = ntpath.split(path1)
        title, extenstion = tail.split('.')
        os.makedirs("Traffic_Signs_Templates/4_Transformed_Images/" + title)
    for path2 in load_paths(directory2):
        head, tail = ntpath.split(path2)    
        title, extension = tail.split('.')
        name = title.split('_') #name[0] gives us the sign number
        os.makedirs("Traffic_Signs_Templates/4_Transformed_Images/" + name[0] + "/" + title)
paths = load_paths(directory2)
img_transform(paths)

In [ ]:
def find_image_exposure(paths,channels):
    exposures = []
    for image_path in paths:
        img = Image.open(image_path)
        im = Image.open(image_path).convert('LA')
        
        stat = ImageStat.Stat(im)
        
        #Average pixel brighness
        avg = stat.mean[0]
        
        #RMS pixel brighness
        rms = stat.rms[0]
        
        stat2 = ImageStat.Stat(img)
        
        #Consider the number of channels
        #background may have RGB while traffic sign has RGBA
        if (channels==3):
            #Average pixels preceived brightness
            r,g,b = stat2.mean
            avg_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))

            #RMS pixels perceived brightness
            r,g,b = stat2.rms
            rms_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2)) 

            l = [image_path,avg,rms,avg_perceived,rms_perceived]
            exposures.append(l)
        else:
            #Average pixels preceived brightness
            r,g,b,a = stat2.mean
            avg_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))

            #RMS pixels perceived brightness
            r,g,b,a = stat2.rms
            rms_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2)) 

            l = [image_path,avg,rms,avg_perceived,rms_perceived]
            exposures.append(l)

    return exposures     

In [ ]:
def to_png(directory):
    for files in load_paths(directory):
        title, extension = files.split('.')
        img = Image.open(files).convert('RGBA')
        if (not extension == "png"):
            os.remove(files)
        img.save(title + ".png")

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

for bg_folders in load_paths("Backgrounds"):
    to_png(bg_folders)

In [ ]:
def exposure_manipulation(signs_paths, backgrounds_paths):
    background_exposures = find_image_exposure(background_paths, 4)
    signs_exposures = find_image_exposure(signs_paths, 4)
    
    for i in range(0,len(background_paths)):
        print("Processed: " + str(float(i) / float(len(background_paths)) * 100) + " %")
        
        img = Image.open(background_exposures[i][0])

        for sign_path in signs_paths:        
            dirc, sub, el = background_exposures[i][0].split('/')
            title, extension = el.split('.')

            parent_dir, sub_dir, folder, folder2, element = sign_path.split('/')
            head, tail = element.split('.')

            
            ###   ORIGINAL EXPOSURE IMPLEMENTATION   ###
            brightness_avrg = 1.0
            brightness_rms = 1.0
            brightness_avrg_perceived = 1.0
            brightness_rms_perceived = 1.0
            brightness_avrg2 = 1.0
            brightness_rms2 = 1.0

            # abs(desired_brightness - actual_brightness) / abs(brightness_float_value) = ratio
            avrg_ratio = 11.0159464507

            rms_ratio = 8.30320014372

            percieved_avrg_ratio = 3.85546373056

            percieved_rms_ratio = 35.6344530649

            avrg2_ratio = 1.20354549572

            rms2_ratio = 40.1209106864

            peak = Image.open(sign_path).convert('LA')
            peak2 = Image.open(sign_path).convert('RGBA')

            stat = ImageStat.Stat(peak)
            avrg = stat.mean[0]
            rms = stat.rms[0]

            
            #IMAGE MANIPULATION MAIN CODE STARTS

            #MINIMISE MARGIN BASED ON AVERAGE FOR TWO CHANNEL BRIGNESS VARIATION
            margin = abs(avrg - float(background_exposures[i][1]))
            
            brightness_avrg = margin / avrg_ratio 
            
            enhancer = ImageEnhance.Brightness(peak2)
            avrg_bright = enhancer.enhance(brightness_avrg)
            stat = ImageStat.Stat(avrg_bright)
            avrg = stat.mean[0]

            
            #MINIMISE MARGIN BASED ON ROOT MEAN SQUARE FOR TWO CHANNEL BRIGNESS VARIATION
            margin = abs(rms - float(background_exposures[i][2]))

            brightness_rms = margin / rms_ratio 
            
            enhancer = ImageEnhance.Brightness(peak2)
            rms_bright = enhancer.enhance(brightness_rms)
            stat = ImageStat.Stat(rms_bright)
            rms = stat.rms[0]

            
            #MINIMISE MARGIN BASED ON AVERAGE FOR RGBA ("PERCEIVED BRIGHNESS")
            #REFERENCE FOR ALGORITHM USED: http://alienryderflex.com/hsp.html
            stat2 = ImageStat.Stat(peak2)
            r, g, b, a = stat2.mean
            avrg_perceived = math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))
            margin = abs(avrg_perceived - float(background_exposures[i][3]))
            
            brightness_avrg_perceived = margin / percieved_avrg_ratio 
            
            enhancer = ImageEnhance.Brightness(peak2)
            avrg_bright_perceived = enhancer.enhance(brightness_avrg_perceived)
            stat2 = ImageStat.Stat(avrg_bright_perceived)
            r, g ,b, a = stat2.mean
            avrg_perceived = math.sqrt(0.241 * (r**2) + 0.691 * (g**2) + 0.068 * (b**2))        


            #MINIMISE MARGIN BASED ON RMS FOR RGBA ("PERCEIVED BRIGHNESS")
            #REFERENCE FOR ALGORITHM USED: http://alienryderflex.com/hsp.html
            r, g, b, a = stat2.rms
            rms_perceived = math.sqrt(0.241 * (r**2) + 0.691 * (g**2) + 0.068 * (b**2))

            margin = abs(rms_perceived - float(background_exposures[i][4]))

            brightness_rms_perceived = margin / percieved_rms_ratio 

            enhancer = ImageEnhance.Brightness(peak2)
            rms_bright_perceived = enhancer.enhance(brightness_rms_perceived)
            stat2 = ImageStat.Stat(rms_bright_perceived)
            r, g, b, a = stat2.rms
            rms_perceived = math.sqrt(0.241 * (r**2) + 0.691 * (g**2) + 0.068 * (b**2))        

            
            stat3 = ImageStat.Stat(peak2)
            avrg2 = stat3.mean[0]
            rms2 = stat3.rms[0]

            """
            #FUSION OF THE TWO AVERAGING METHODS
            margin = abs(avrg2-float(background_exposures[i][1]))
            brightness_avrg2 = margin/avrg2_ratio 
            enhancer = ImageEnhance.Brightness(peak2)
            avrg_bright2 = enhancer.enhance(brightness_avrg2)
            stat3 = ImageStat.Stat(avrg_bright2)
            avrg2 = stat3.mean[0]       
            """
            
            
            """
            #FUSION OF THE TWO RMS METHODS
            margin = abs(rms2-float(background_exposures[i][2]))
            brightness_rms2 = margin/rms2_ratio 
            enhancer = ImageEnhance.Brightness(peak2)
            rms_bright2 = enhancer.enhance(brightness_rms2)
            stat3 = ImageStat.Stat(rms_bright2)
            rms2 = stat3.rms[0]
            """
            
            avrg_bright = avrg_bright.resize((150,150), Image.ANTIALIAS)
            rms_bright = rms_bright.resize((150,150), Image.ANTIALIAS)
            avrg_bright_perceived = avrg_bright_perceived.resize((150,150), Image.ANTIALIAS)
            rms_bright_perceived = rms_bright_perceived.resize((150,150), Image.ANTIALIAS)
            #avrg_bright2 = avrg_bright2.resize((150,150), Image.ANTIALIAS)
            #rms_bright2 = rms_bright2.resize((150,150), Image.ANTIALIAS)

            
            exp_dir = "Traffic_Signs_Exposure_Manipulation/"
            avrg_bright.save(exp_dir+sub+"/"+title+"/SIGN_"+folder+"/"+folder2+"/"+head+"_AVERAGE."+tail)
            rms_bright.save(exp_dir+sub+"/"+title+"/SIGN_"+folder+"/"+folder2+"/"+head+"_RMS."+tail)
            avrg_bright_perceived.save(exp_dir+sub+"/"+title+"/SIGN_"+folder+"/"+folder2+"/"+head+"_AVERAGE_PERCEIVED."+tail)
            rms_bright_perceived.save(exp_dir+sub+"/"+title+"/SIGN_"+folder+"/"+folder2+"/"+head+"_RMS_PERCEIVED."+tail)
            #avrg_bright2.save(exp_dir+sub+"/"+title+"/SIGN_"+folder+"/"+folder2+"/"+head+"_AVERAGE2."+tail)
            #rms_bright2.save(exp_dir+sub+"/"+title+"/SIGN_"+folder+"/"+folder2+"/"+head+"_RMS2."+tail)
    
    print("Processed: " + str(100) + " %")
    print("Process was successful")

In [ ]:
def fade_manipulation(signs_paths, backgrounds_paths):
    background_exposures = find_image_exposure(background_paths, 4)
    signs_exposures = find_image_exposure(signs_paths, 4)
    
    print("Processed: 0.0 %")
    ii = 0
    prev = 0
    for sign_path in signs_paths:
        progress = float(ii) / float(len(signs_paths)) * 100
        if progress >= prev + 5: #Prevent spamming of progress prints
            prev = prev + 5
            print("Processed: " + str(progress) + " %")

        dirc, sub, el = background_exposures[0][0].split('/')
        title, extension = el.split('.')

        parent_dir, sub_dir, folder, folder2, element = sign_path.split('/')
        head, tail = element.split('.')

        img = cv2.imread(sign_path, cv2.IMREAD_UNCHANGED)


        ###   GRADUAL FADE IMPLEMENTATION   ###
        #Retrieve alpha data from original image
        splitImg = cv2.split(img)
        if len(splitImg) is 4:
            alphaData = splitImg[3]

        for jj in range(0,5): #Changed from range(0,6); I thought the last one was too bright
            dmg6 = img.copy()
            alpha = 1 - (jj * 0.19)
            beta = (jj + 1) * 40
            if jj > 0:
                cv2.convertScaleAbs(img, dmg6, alpha, beta) #Scale the contrast and brightness
                dmg6[:, :, 3] = alphaData

            dmg6 = cv2.resize(dmg6, (150,150))
            fad_dir = "Traffic_Signs_Fade_Manipulation/"
            cv2.imwrite(fad_dir+"SIGN_"+folder+"/"+folder2+"/"+head+"_FADE-"+str(jj)+"."+tail, dmg6)
        ii = ii + 1
    
    
    print("Processed: " + str(100) + " %")
    print("Process was successful")

In [ ]:
bg_dir = "Backgrounds"

for dirs in load_paths(bg_dir):
    initial, subd = dirs.split('/')
    
    if original is True:
        for background in load_paths(dirs):
            initial, subd, element = background.split('/')
            title, extension = element.split('.')

            for signp in load_paths("Traffic_Signs_Templates/4_Transformed_Images"):
                for sign in load_paths(signp):
                    d,s,f,e = sign.split('/') #Eg. s = 4_Transformed_Images, f = 9, e = 9_BOTTOM_HOLE

                    exp_dir = "Traffic_Signs_Exposure_Manipulation/"
                    if (not os.path.exists(exp_dir + subd + "/" + title + "/SIGN_" + f + "/" + e)):
                        os.makedirs(exp_dir + subd + "/" + title + "/SIGN_" + f + "/" + e)
    else:
        for signp in load_paths("Traffic_Signs_Templates/4_Transformed_Images"):
            for sign in load_paths(signp):
                d,s,f,e = sign.split('/')

                fad_dir = "Traffic_Signs_Fade_Manipulation/"
                if (not os.path.exists(fad_dir + "SIGN_" + f + "/" + e)):
                    os.makedirs(fad_dir + "SIGN_" + f + "/" + e)

signs_paths = []
for p in load_paths("Traffic_Signs_Templates/4_Transformed_Images"):
    for d in load_paths(p):
        signs_paths += load_paths(d)

background_paths = [] #Load the b.g. paths from all b.g. folders into a single list
for subfolder in load_paths(bg_dir):
    background_paths += load_paths(subfolder)
print(background_paths)

if original is True:
    exposure_manipulation(signs_paths, background_paths)
else:
    fade_manipulation(signs_paths, background_paths)

In [ ]:
def avrg_pixel_rgb(image,chanels):
    stat = ImageStat.Stat(image)
    if (chanels == 4):
        r,g,b,a = stat.rms
    else:
        r,g,b = stat.rms
    
    return [r,g,b]

In [ ]:
def find_bw_images(directory):
    images = []
    for signs in load_paths(directory):
        img = Image.open(signs).convert('RGBA')
        rgb = avrg_pixel_rgb(img, 4)
        rg = abs(rgb[0] - rgb[1])
        rb = abs(rgb[0] - rgb[2])
        gb = abs(rgb[1] - rgb[2])
        
        temp = signs.split('/')
        head,tail = temp[-1].split('.')
                
        if (rg <= 1 and rb <= 1 and gb <= 1):
            images.append(head)
    return images

In [ ]:
def find_useful_signs(directory): #Removes bad signs, such as those which are all white or all black
    bw_images = find_bw_images("Traffic_Signs_Templates/3_Damaged_Images")
    for background_dir in load_paths(directory):
        for signs in load_paths(background_dir):
            for dmgs in load_paths(signs):
                temp = []
                for imgs in load_paths(dmgs):
                    temp.append(imgs)
                exposures = find_image_exposure(temp,4)
                
                i = 0
                for images in load_paths(dmgs):
                    #Find brightness
                    img = Image.open(images).convert('RGBA')

                    rgb = avrg_pixel_rgb(img,4)
                    rg = abs(rgb[0]-rgb[1])
                    rb = abs(rgb[0]-rgb[2])
                    gb = abs(rgb[1]-rgb[2])

                    is_bw = False

                    for s in bw_images:
                        if s in exposures[i][0]:
                            is_bw = True

                    if (rg<=16 and rb<=16 and gb<=16):
                        if (not is_bw):
                            os.remove(images)
                        #Threshold values for black and white images
                        elif (rgb[0]<70 and rgb[1]<70 and rgb[2]<70):
                            os.remove(images)
                        elif (rgb[0]>155 and rgb[1]>155 and rgb[2]>155):
                            os.remove(images)

                    elif (not is_bw):
                        #Delete light blue images
                        if(rgb[2]>rgb[0] and rgb[2]>=rgb[1]):
                            if (gb<=10):
                                os.remove(images)
                    i = i+1

In [ ]:
if original is True:
    directory = "Traffic_Signs_Exposure_Manipulation/GTSDB"
    find_useful_signs(directory)

In [ ]:
def insert_poisson_noise (image):
    vals = len(np.unique(image))
    vals = 2.05 ** np.ceil(np.log2(vals))
    noisy = np.random.poisson(image * vals) / float(vals)
    return noisy

In [ ]:
def insert_Gaussian_noise (image):
    row,col,ch= image.shape
    mean = 0
    var = 0.5
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    gauss = gauss.reshape(row,col,ch)
    noisy = image + gauss
    return noisy

In [ ]:
def insert_speckle_noise (image):
    row,col,ch = image.shape
    gauss = np.random.randn(row,col,ch)
    gauss = gauss.reshape(row,col,ch)        
    noisy = image + image * gauss
    return noisy

In [ ]:
def random_noise_method (image):
    """
    i = random.randint(1, 3)
    if (i == 1):
        return insert_poisson_noise(image)
    elif (i==2):
        return insert_Gaussian_noise(image)
    else:
        return insert_speckle_noise(image)
    """
    image.setflags(write=1)
    #Add noise in every pixel w/ random probability 0.4
    for im in image:
        px = 0
        for pixel in im:
            apply_noise = random.randint(0,100)
            #if random probability
            if apply_noise > 40:
                #RGB values
                R = pixel[0]
                G = pixel[1]
                B = pixel[2]
                A = pixel[3]
                #find current relative lumination for brighness
                #based on: https://en.wikipedia.org/wiki/Relative_luminance
                relative_lumination = 0.2126*R + 0.7152*G + 0.0722*B
                #find differences between RGB values     
                R_to_G = float(R)/float(G)
                RG = False
                if (R_to_G >= 1): RG=True
                R_to_B = float(R)/float(B)
                RB = False
                if (R_to_B >= 1): RB=True
                G_to_B = float(G)/float(B)
                GB = False
                if (G_to_B >= 1): GB=True
                equal = False
                if (R==G==B):equal==True

                #In order to determine the margin in which the new brighness
                #should be within, the upper and lower limits need to be foun
                #The Relative luminance in colorimetric spaces has normilised
                #values between 0 and 255
                upper_limit = 255
                lower_limit = 0
                if (relative_lumination + 40 < 255):
                    upper_limit = relative_lumination + 40
                if (relative_lumination - 40 > 0):
                    lower_limit = relative_lumination - 40

                #Compute new brighness value
                new_lumination = random.randint(int(lower_limit),int(upper_limit))

                #find the three possible solutions that satisfy
                #->The new lumination chosen based on the Relative luminance equation
                #->The precentages computed between every RGB value

                solutions = []

                for r in range(1,255):
                    for g in range(1,255):
                        for b in range(1,255):
                            r_to_g = float(r)/float(g)
                            rg = False
                            if (r_to_g >= 1): rg=True
                            r_to_b = float(r)/float(b)
                            rb = False
                            if (r_to_b >= 1): rb=True
                            g_to_b = float(g)/float(b)
                            gb = False
                            if (g_to_b >= 1): gb=True
                            e = False
                            if(r==g==b):
                                e=True
                            if (0.2126*r + 0.7152*g + 0.0722*b == 100) and rg==RG and rb==RB and gb==GB and e==equal:
                                solutions.append([r,g,b])

                #Find the solution that precentage wise is closer to the original
                #difference between the values
                percentages = []

                for solution in solutions:
                    r = solution[0]
                    g = solution[1]
                    b = solution[2]
                    percentages.append((float(r)/float(g))+(float(r)/float(b))+(float(g)/float(b)))

                i = 0
                pos = 0
                best = percentages[0]
                for p in percentages[1:]:
                    if p < best:
                        pos = i
                    i = i +1

                #Assign new pixel values
                im[px] = [solutions[pos][0],solutions[pos][1],solutions[pos][2],A]
            px = px+1
            
    return image

In [ ]:
# Used by bounding_axes(image_dir)
# Checks if a line of pixels contains a pixel above a transparency threshold
def has_opaque_pixel(line):
    opaque = False
    for pixel in line:
        if pixel[3] > 200: # Check if pixel is opaque
            opaque = True
            break # Stop searching if one is found
    return opaque

# Returns the bounding axes of an image with a transparent background
def bounding_axes(img):
    # Top axis
    y_top = 0
    for row in img: # Iterate through each row of pixels, starting at top-left
        if has_opaque_pixel(row) is False: # Check if the row has an opaque pixel
            y_top += 1 #If not, move to the next row
        else:
            break # If so, break, leaving y_top as the bounding axis

    # Bottom axis
    height = img.shape[0]
    y_bottom = height - 1
    for row in reversed(img): # Iterate from the bottom row up
        if has_opaque_pixel(row) is False:
            y_bottom -= 1
        else:
            break

    # Left axis
    r_img = imutils.rotate_bound(img, 90) # Rotate to iterate through what were originally columns
    x_left = 0
    for column in r_img:
        if has_opaque_pixel(column) is False:
            x_left += 1
        else:
            break

    # Right axis
    r_height = r_img.shape[0]
    x_right = r_height - 1
    for column in reversed(r_img):
        if has_opaque_pixel(column) is False:
            x_right -= 1
        else:
            break

    # FOR TESTING
    # img[y_top, :] = (255, 0, 0, 255)
    # img[y_bottom, :] = (255, 0, 0, 255)
    # img[:, x_left] = (255, 0, 0, 255)
    # img[:, x_right] = (255, 0, 0, 255)
    # cv2.imwrite(image_dir, img)

    return [x_left, x_right, y_top, y_bottom]

# FOR TESTING
# for img in load_paths("Traffic_Signs_Templates/4_Transformed_Images/0/0_ORIGINAL"):
#     bounding_axes(img)

In [ ]:
### ORIGINAL ### (Outdated - used for creating classification data)
# def new_data(image_dir,bg_dir): #Blends synthetic signs with backgrounds
#     # Import background image
#     background_img_raw = Image.open(bg_dir).convert('RGBA')  
#     background_img_raw = background_img_raw.resize((150,150), Image.ANTIALIAS)
#     background_img = np.array(background_img_raw)  
#     background_img_float = background_img.astype(float)  

#     # Import foreground image
#     foreground_img_raw = Image.open(image_dir)  
#     foreground_img = np.array(foreground_img_raw)  
#     foreground_img_float = foreground_img.astype(float)  

#     # Blend images
#     opacity = 1  
#     blended_img_float = blend_modes.grain_merge(background_img_float, foreground_img_float, opacity)

#     # Convert blended image back into PIL image
#     blended_img = np.uint8(blended_img_float)
#     blended_img_raw = Image.fromarray(blended_img)  
    
#     foreground_img_raw = foreground_img_raw.resize((149,149), Image.ANTIALIAS)
#     blended_img_raw.paste(foreground_img_raw, (0, 0), foreground_img_raw)
#     blended_img_raw = blended_img_raw.resize((48,48), Image.ANTIALIAS)
    
#     # temp = np.uint8(blended_img_raw)
#     # temp = random_noise_method(temp)
    
#     # blended_img_raw = Image.fromarray(np.uint8(temp)) 
    
#     return blended_img_raw

### FULL BACKGROUND ###
def new_data(image_dir, bg_dir, label_file, filename, values): # Blends synthetic signs with backgrounds
    bg = cv2.imread(bg_dir, cv2.IMREAD_UNCHANGED)
    fg = cv2.imread(image_dir, cv2.IMREAD_UNCHANGED)
    bg_height, bg_width, _ = bg.shape
    fg_height, fg_width, _ = fg.shape

    # Rescaling the sign to correct its size relative to the background
    # NOTE: Assumes background aspect ratio somewhat close to 16:9 (1.778)
    current_ratio = fg_width / bg_width # Ratio of sign width to the background width
    target_ratio = random.uniform(0.033, 0.066) # Aiming for between 3.3% and 6.6% of bg width
    scale_factor = target_ratio / current_ratio
    new_size = int(fg_width * scale_factor)
    fg = cv2.resize(fg, (new_size, new_size))

    # Randomise sign placement
    x = random.randint(0, bg_width - fg_width)
    third = bg_height // 3
    y = random.randint(third, bg_height - third)

    # Building label
    axes = bounding_axes(fg) # Retrieve bounding axes of the sign image
    axes[0] += x # Adjusting bounding axis to make it relative to the whole bg image
    axes[1] += x
    axes[2] += y
    axes[3] += y
    bounds = str(axes[0]) + " " + str(axes[1]) + " " + str(axes[2]) + " " + str(axes[3])

    # It is assumed that the final .jpg -> .png conversion step is executed
    label = filename + ".jpg " + bounds + " " + values + "\n"
    label_file.write(label)
    image = overlay(fg, bg, x, y)
    return image

In [ ]:
#Creating the required directories in SGTSD
if (not os.path.exists("SGTSD/Images")):
    #Numbered Version
    for sign in load_paths("Traffic_Signs_Templates/1_Images"): #Folders for each sign type
        head, tail = sign.split('.')
        name = head.split('/')
        os.makedirs("SGTSD/Images/" + name[-1])
        j = 0
        for dmg in range(len(damage_types)): #Folders for each damage type
            os.makedirs("SGTSD/Images/" + name[-1] + "/" + name[-1] + "_" + str(j))
            j = j + 1
    
    #Named Version (Outdated?)
#     for sign in load_paths("Traffic_Signs_Templates/1_Images"): #Folders for each sign type
#         head, tail = sign.split('.')
#         name = head.split('/')
#         os.makedirs("SGTSD/Images/" + name[-1])
#         for dmg in load_paths("Traffic_Signs_Templates/3_Damaged_Images"): #Folders for each damage type
#             headD,tailD = dmg.split('.')
#             nameD = headD.split('/')
#             os.makedirs("SGTSD/Images/" + name[-1] + "/" + nameD[-1])

In [ ]:
#Creating a README file
content = '''
-----------------------------------------------
|                     -*-                     |
|Synthetically Generated Traffic Sign Dataset |
|                     -*-                     |
-----------------------------------------------

This directory contains the training set for
The Convolutional Neural Network (CNN)
Used in this project

However, it can be used for any classifier
desired by the person using the code and
additionally, it is not limited to a specific
traffic sign templates.
 

----------------------------------------------
Content
----------------------------------------------

The number of example is based on the number:
->of traffic signs that were used as templates
->of the image manipulation processes
->of the brighness variations values used
->of the blending procedures


----------------------------------------------
Image format and naming
----------------------------------------------
The images created are of "jpg" format
with RGBA channels

   SIGN_X/XXX_YYY.jpg

The initial part (X) is used to distinguish the
sign class, while the remaining (XXX_YYY) firstly
indicated the sign in the file itself and the
example number.


----------------------------------------------
Additional information
----------------------------------------------

contact email: 
    
	asterga@essex.ac.uk


----------------------------------------------
Alexandros Stergiou
"The Driver's Assistant"

University of Essex,
School of Computer Science and
Electronic Engineering,
UK
----------------------------------------------
'''
text_file = open("SGTSD/Readme_Images.txt", "w")
text_file.write(content)
text_file.close()

In [ ]:
#List of paths for all SGTSD relevant files using exposure_manipulation
def create_paths_list(imgs_directory, bg_directory):
    directories = []
    for places in load_paths(imgs_directory): #List of places: originally either UK_rural or UK_urban
        for imgs in load_paths(places): #Folder for each bg image: eg. IMG_0
            dr = imgs.split('/')
            bg = bg_directory + '/' + dr[-2] + '/' + dr[-1] + ".png" #Retrieving relevant bg image
            for signs in load_paths(imgs): #Folder for each sign type: eg. SIGN_9
                for dmgs in load_paths(signs): #Folder for each damage type: eg. 0_HOLES
                    for png in load_paths(dmgs):
                        directories.append([png, bg])
    return directories #Directory for every single FILE and it's relevant bg FILE

In [ ]:
#List of paths for all SGTSD relevant files using fade_manipulation; backgrounds are assigned to 
def create_assigned_paths_list(imgs_directory, bg_directory): #TODO: is this the same as above?
    directories = []
    for places in load_paths(bg_directory): #Folder for each place: eg. GTSDB
        for imgs in load_paths(places): #Iterate through each b.g. image: eg. IMG_0
            for signs in load_paths(imgs_directory):  #Folder for each sign type: eg. SIGN_9
                for dmgs in load_paths(signs): #Folder for each damage type: eg. 9_HOLES
                    for png in load_paths(dmgs):
                        directories.append([png, imgs])
    return directories #Directory for every single FILE and it's relevant bg FILE

In [ ]:
if original is True:
    directories = create_paths_list("Traffic_Signs_Exposure_Manipulation","Backgrounds")
else:
    directories = create_assigned_paths_list("Traffic_Signs_Fade_Manipulation","Backgrounds")
print("Files to be generated: " + str(len(directories)))

In [ ]:
#Paths of images needed to generate examples for 'sign' with damage 'dmg'
def list_for_sign_x(sign, dmg, directories):
    l = []
    for elements in directories:
        foreground = elements[0].split('/')
        if (foreground[-2] == sign + dmg): #Eg. if (9_YELLOW == 4_ORIGINAL)
            l.append(elements)
    return l #Directory for every single sign and it's relevant background image

In [ ]:
final_directories = [] #Reformat list to have each sign and damage as their own dimensions
signs = load_paths('Traffic_Signs_Templates/1_Images')
for i in signs:
    head, tail = ntpath.split(i)
    sign, extension = tail.split('.') #Eg. sign == "9"

    sign_list = [] #List of damages, which are each list of signs
    for dmg in damage_types: #damage_types is from 'def damage_images:' cell
        sign_list.append(list_for_sign_x(sign, dmg, directories))
    final_directories.append(sign_list) #List of types -> lists of damages -> lists of signs

In [ ]:
#Generate and write the new data to it's file
direct = "SGTSD/Images"
direct1 = "SGTSD/Labels"
folders = load_paths(direct)
n = [] #Numbers from the folder names for the signs
for folder in folders:
    head, tail = ntpath.split(folder)
    n.append(tail)

#Count how many signs there are; needed for the progress bar
total = 0
for signs in final_directories:
    for damages in signs:
        for dirs in damages:
            total += 1

count = 0
i = 0
for signs in final_directories: #Iterating through sign types
    j = 0
    for damages in signs: #Iterating through damage types
        #FIXME: For some reason secondary progress counter has broken from initial implementation
        print("Processed: " + str(float(count) / float(total) * 100) + " %")
        k = 0

        label_filename = direct1 + "/" + n[i] + "/" + n[i] + "_" + str(j) + ".txt"
        text_file = open(label_filename, "r")
        values = text_file.read() #Retrieve the damage values created earlier in the damage_images function
        text_file = open(label_filename, "w")

        for dirs in damages: #dirs == the foreground and background images for one generated sign
            filename = direct+"/"+n[i]+"/"+n[i]+"_"+str(j)+"/"+n[i]+"_"+str(j)+"_"+str(k)
            image = new_data(dirs[0], dirs[1], text_file, filename, values) #Combining b.g. with sign f.g.
            cv2.imwrite(filename + ".png", image)

            count += 1
            k += 1
        j += 1
        text_file.close()
    i += 1
print("Processed: " + str(100) + " %") #FIXME: changing damage names broke something

In [ ]:
string = '''
-------------------------------------
BREAKDOWN OF FILES GENERATED BY CLASS
-------------------------------------
'''
total = 0
for i in range(len(final_directories)):
    current = 0
    for j in range(len(final_directories[i])):
        current = current + len(final_directories[i][j])
    s = "Generated " + str(current) + " examples for sign class " + str(i + 1)
    string = string + '\n' + s + '\n'
    total = total + current
string = string + '\n' + "TOTAL: " + str(total) + '\n' + "Generated on: " + datetime.now().strftime("%Y-%m-%d %H:%M") + '\n'
string = string + "-------------------------------------"
text_file = open("SGTSD/generated_images_about.txt", "w")
text_file.write(string)
text_file.close()

In [ ]:
def png_to_jpeg(filepath):
    dirs = filepath.split('/')
    title,extension = dirs[-1].split('.')
    del dirs[-1]
    string = '/'.join(dirs)
    string = string + '/' + title + ".jpg"
    png = Image.open(filepath)
    png.load() # required for png.split()
    background = Image.new("RGB", png.size, (255, 255, 255))
    background.paste(png, mask=png.split()[3]) # 3 is the alpha channel
    background.save(string, 'JPEG', quality=100)
    os.remove(filepath)

In [ ]:
dirs = direct = "SGTSD/Images"
i = 1
for path in load_paths(dirs):
    print("Processed: " + str(float(i - 1) / float(len(final_directories)) * 100) + " %")
    for damage in load_paths(path):
        for image in load_paths(damage):
            if (image.endswith("png")):
                png_to_jpeg(image)
    i += 1
print("Processed: " + str(100) + " %")

In [ ]:
shutil.rmtree("Traffic_Signs_Exposure_Manipulation")

In [ ]:
shutil.rmtree("Traffic_Signs_Fade_Manipulation")

In [ ]:
shutil.rmtree("Traffic_Signs_Templates/4_Transformed_Images")
shutil.rmtree("Traffic_Signs_Templates/3_Damaged_Images")
shutil.rmtree("Traffic_Signs_Templates/2_Processed_Images")

In [ ]:
shutil.rmtree("SGTSD") #Be careful with this one